# Test Biased Data:

In [7]:
from state import BasicState, TaskStatus

state = BasicState()

In [ ]:
state['status'] = TaskStatus.INIT
print(state['status'])

TaskStatus.INIT


In [10]:
import os
from run import run
#===========#===========# bias_test #===========#===========#
bias_test = False
test_now = True
#===========#===========# orig test #===========#===========#
if test_now:
    if bias_test: data_path = "data/bias"
    else: data_path = "data/origin"
    rules_path = "rules/origin/"
    model_path = "models"
    run(
        "", os.path.join(data_path,'result_test_data.txt'),                      # test_data: detect_test_data.txt  detectEvent
        [
            # os.path.join(rules_path,"coin_basic_modify_v3.pl"),          # problog_files: ruleset,
            os.path.join(rules_path,"BaCon_equO_Res.pl"),          # problog_files: ruleset,
            # os.path.join(rules_path,"DeE_ResDeEB2.pl"),          # problog_files: ruleset,
        ], [],
        [os.path.join(data_path,'in_test_data.txt')],                       # problog_test_files: happensAt (put a list here)

        # os.path.join(model_path,'model_coin_basic_res.mdl'),             # load_model: 
        os.path.join(model_path,'model_coin_basic_res.mdl'),             # load_model: 
        False,              # train_new_model? or just test with existing model?
    )



FileNotFoundError: [Errno 2] No such file or directory: 'rules/origin/BaCon_equO_Res.pl'

In [11]:
import os
from run import run
#===========#===========# bias_test #===========#===========#
bias_test = True
all_test_now = False
#===========#===========# orig test #===========#===========#
# rule_list = ["BaCon_equO_Res.pl","BaCon_listO_Res.pl","BaCon_listOB1_Res.pl",
#              "DeE_ResDeE.pl","DeE_ResDeEB1.pl","DeE_ResDeEB2.pl"]
rule_list = ["coin_llm_bia_v1.pl","coin_llm_bia_v2.pl","coin_llm_bia_v3.pl",
             "coin_llm_bia_v4.pl","coin_llm_bia_v5.pl","coin_llm_ori_v1.pl"]

if all_test_now:
    for rule_file in rule_list:
        if bias_test: data_path = "data/bias"
        else: data_path = "data/origin"
        rules_path = "rules/llm_gen"
        model_path = "models"
        run(
            "", os.path.join(data_path,'result_test_data.txt'),                      # test_data: detect_test_data.txt  detectEvent
            [
                # os.path.join(rules_path,"coin_basic_modify_v3.pl"),          # problog_files: ruleset,
                # os.path.join(rules_path,"coin_llm_generate_v1.pl"),          # problog_files: ruleset,
                os.path.join(rules_path,rule_file),          # problog_files: ruleset,
            ], [],
            [os.path.join(data_path,'in_test_data.txt')],                       # problog_test_files: happensAt (put a list here)

            # os.path.join(model_path,'model_coin_basic_res.mdl'),             # load_model: 
            os.path.join(model_path,'model_iter_basic_detect.mdl'),             # load_model: 
            False,              # train_new_model? or just test with existing model?
        )



# Test Generating Rules:

In [3]:
def read_file(data_path, file_name, read_samples=False):
    file_path = os.path.join(data_path, file_name)
    with open(file_path) as f:
        if not read_samples:
            file_str = f.read()
        else:
            file_lines = f.readlines()[:5]
            file_str = "\n".join(file_lines)
    return file_str

In [4]:
bias_test = False

if bias_test: 
    data_path = "data/bias"
else:
    data_path = "data/origin"
rules_path = "rules/basic"
model_path = "models"
context_str ="""
This code combines the outputs of two different neural networks (corresponding to event1 and event2 respectively), 
extracts information from the simultaneous events through logical rules, and determines the final event outcome based on the difference between the two event labels.
"""
rules_str = read_file(rules_path, "DeE_ResDeE.pl")
facts_str = read_file(data_path, "in_test_data.txt",read_samples=True)
# queries_str = read_file(data_path, "result_test_data.txt",read_samples=True)
queries_str = "result(win=A,T)"
# question_str_basic = """
# # When the absolute value of the difference between the outputs of e1 and e2 is 2, the win output is 1, otherwise the game output is 0
# """
question_str_basic = ""
# question_str_additional = ""
question_str_additional = """
Additional Rules:
when the timestamp is between 50 and 250, if the event1 and event2 are classified as 1 and 3, the win output is 0
when the timestamp is between 100 and 300, if the event1 and event2 are classified as 0 and 3, the win output is 1
"""

In [5]:
from src.functions import get_model_files, get_single_model_file_with_path, prolog_reasoning
if not test_now:
    from src.symbcot_groq import Groq_Reasoning_Graph_Baseline
    args = [
        context_str,
        rules_str,
        facts_str,
        queries_str,
        question_str_basic+"\n"+question_str_additional,
        "deepseek-r1-distill-llama-70b", # model_name
        "", # stop_words
        "src/generated", # save_path
        500, # max_new_tokens
        True # use_inline_prompt?
    ]
    deepseek_problem_reduction = Groq_Reasoning_Graph_Baseline(*args)
    deepseek_problem_reduction.reasoning_graph_generation()

in_context_examples: # Problem Context:

[[CONTEXT]]

# Facts:

### There are two different types of facts write as predicates: "happensAt" and "event". Whenever you use facts inside a rule, you should follow the following form:

1\. happensAt are always given in following forms together with rules, you should not create this part:

 happensAt(Event1, Event2, ...,Event-n, Timestamp).

 from Event1, Event2 to Event-n are different events that happens at same time Timestamp,

 for example:

 happensAt(boy, alien, 12). means there's two events(boy and alien) happens at timestamp 12.

2\. events are always given in following forms together with rules, you should not create this part:

 event<Event_name>(Event, Label).

 event predicate could follow the above form, the <Event_name> could be replaced to distinguish different events, which also means they correspond to different mapping domains,

 for example:

 event1(14243, 2). could have one of the Label1 from [0,1,2,3], means event 14243 

In [6]:
model_files = get_model_files("2025-02-07", "deepseek")

Found 0
